In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..", "src")))


import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn
from tqdm import tqdm

from sklearn import cross_decomposition
from vip import *
from np_classifier import *

tqdm.pandas()

Define filepaths

In [12]:
rise_metadata_path = "../data-raw/mzmine_data_from_jasmine/Plusrise_Metadata_abstract.csv"
rise_quant_table_path = "../data-raw/mzmine_rerun/rise_plus_filtered_quant_table_rerun.csv"
annotations_path = "../data-raw/mzmine_rerun/MSV000096359_run1_output_spectral_annotations.csv"

rise_patient_matches_path = "../data-out-unclean/matched_patients_without_replacement.csv"


Compute VIP Scores for Molecules

In [43]:
# Load and preprocess metadata
metadata = pd.read_csv(rise_metadata_path, sep=",")[["ATTRIBUTE_participantid", "ATTRIBUTE_Metabolomics_ID"]]
patient_to_sample = dict(zip(metadata["ATTRIBUTE_participantid"], metadata["ATTRIBUTE_Metabolomics_ID"]))
sample_to_patient = {v: k for k, v in patient_to_sample.items()}

# Load metabolomics data
metabolomics_data = pd.read_csv(rise_quant_table_path, sep=",", index_col=0)
feature_metadata = metabolomics_data.iloc[:, :13]
sample_data = metabolomics_data.iloc[:, 13:].drop(columns=["Unnamed: 774"])
annotation_data = pd.read_csv(annotations_path)


# Extract group info and rename columns to sample IDs
groups = np.array([col.split("_")[0] for col in sample_data.columns])
sample_data.columns = [col.split(".")[0].split("_")[2] for col in sample_data.columns]

# Load patient matching data and filter for high-quality matches
patient_matches = pd.read_csv(rise_patient_matches_path, sep=",", index_col=0)
cancer_patients = patient_matches[
    (patient_matches["cancer"] == True) & 
    (patient_matches["absolute_propensity_difference"] < 0.01)
]
control_patients = patient_matches.loc[cancer_patients["nearest_patient"]].index.tolist()
all_study_patients = cancer_patients.index.tolist() + control_patients

# Filter metabolomics data for study patients
study_sample_ids = [str(patient_to_sample[patient]) for patient in all_study_patients]
study_data = sample_data.loc[:, sample_data.columns.isin(study_sample_ids)]
study_data.columns = [sample_to_patient[int(col)] for col in study_data.columns]

# Remove features with zero total abundance
study_data = study_data[study_data.sum(axis=1) > 0]

# Prepare data for PLS regression (transpose: samples as rows, features as columns)
data_t = study_data.T
data_t["is_cancer"] = data_t.index.isin(cancer_patients.index)

# Create binary target matrix for PLS
y_cancer = data_t["is_cancer"].astype(int).values.reshape(-1, 1)
y_control = (1 - y_cancer).reshape(-1, 1)
y_pls = np.hstack([y_cancer, y_control])

# Fit PLS regression
pls_model = cross_decomposition.PLSRegression(n_components=2)
pls_output = pls_model.fit_transform(data_t.iloc[:, :-1].values, y_pls)

# Apply CLR (Centered Log-Ratio) transformation
data_normalized = data_t.copy()
feature_data = data_normalized.iloc[:, :-1]

# Step 1: Convert to relative abundances
feature_data = feature_data.div(feature_data.sum(axis=1), axis=0)

# Step 2: Divide by geometric mean (ignoring zeros)
geom_means = np.exp(np.mean(np.log(feature_data.replace(0, np.nan)), axis=1))
feature_data = feature_data.div(geom_means, axis=0)

# Step 3: Handle zeros and apply log transformation
min_vals = feature_data.replace(0, np.nan).min(axis=1)
feature_data = feature_data.apply(lambda row: row.replace(0, min_vals[row.name]), axis=1)
feature_data = np.log(feature_data)

# Update normalized data
data_normalized.iloc[:, :-1] = feature_data

# Calculate VIP scores
vip_scores = vip(data_normalized.iloc[:, :-1].values, y_pls, pls_model)

# Merge VIP Scores with feature metadata

row_IDs = dict(zip(feature_metadata["row ID"].index.to_list(), feature_metadata["row ID"].values.tolist()))

annotation_data = annotation_data.loc[annotation_data.groupby("id")["score"].idxmax()]
feature_dict = dict(zip(annotation_data["id"].values.tolist(), annotation_data["compound_name"].values.tolist()))
scores_dict = dict(zip(annotation_data["id"].values.tolist(), annotation_data["score"].values.tolist()))
smiles_dict = dict(zip(annotation_data["id"].values.tolist(), annotation_data["smiles"].values.tolist()))
adduct_dict = dict(zip(annotation_data["id"].values.tolist(), annotation_data["adduct"].values.tolist()))
rt_dict = dict(zip(metabolomics_data["row ID"].values.tolist(), metabolomics_data["row retention time"].values.tolist()))

vip_dataframe = pd.DataFrame(index=data_normalized.columns[:-1], data=vip_scores, columns=["VIP Score"])
vip_dataframe["row ID"] = vip_dataframe.index.map(row_IDs)
vip_dataframe["retention time"] = vip_dataframe["row ID"].map(lambda x: rt_dict.get(x, None))
vip_dataframe["adduct"] = vip_dataframe["row ID"].map(lambda x: adduct_dict.get(x, None))
vip_dataframe["compound name"] = vip_dataframe["row ID"].map(feature_dict)
vip_dataframe["compound score"] = vip_dataframe["row ID"].map(scores_dict)
vip_dataframe["smiles"] = vip_dataframe["row ID"].map(lambda x: smiles_dict.get(x, None))



Classify compounds with NP Classifier

In [45]:
# Get all entries that appear multiple times
vip_dataframe["count"] = vip_dataframe.groupby("smiles")["smiles"].transform("count")
vip_dataframe = vip_dataframe[vip_dataframe["count"] > 1]

In [47]:
vip_dataframe["VIP Score"] = vip_dataframe["VIP Score"].astype(float).round(3)

/var/folders/7f/td1j1ghj1f77gp6tfcmnv2lw0000gn/T/ipykernel_60038/2578306921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vip_dataframe["VIP Score"] = vip_dataframe["VIP Score"].astype(float).round(3)


In [ ]:
vip_dataframe

,VIP Score,row ID,retention time,adduct,compound name,compound score,smiles,count
3,1.640,5533,1.302363,[M-H2O+H]+,5_AMINOPENTANOATE,0.909,NCCCCC(O)=O,5.0
12,1.114,5373,1.303793,[M-H2O+H]+,5_AMINOPENTANOATE,0.889,NCCCCC(O)=O,5.0


In [ ]:
np_classifier_classifications = vip_dataframe["smiles"].progress_apply(lambda x: get_npclassifier(x) if x is not None else None)
``

In [28]:
vip_dataframe["class"] = np_classifier_classifications.apply(lambda x : x["class"] if x is not None and "class" in x else None)
vip_dataframe["superclass"] = np_classifier_classifications.apply(lambda x : x["superclass"] if x is not None and "superclass" in x else None)
vip_dataframe["pathway"] = np_classifier_classifications.apply(lambda x : x["pathway"] if x is not None and "pathway" in x else None)
vip_dataframe["is_glycoside"] = np_classifier_classifications.apply(lambda x: x["is_glycoside"] if x is not None and "is_glycoside" in x else None)

In [33]:
vip_dataframe_annotated = vip_dataframe.dropna(subset=["compound score"])
vip_dataframe_unique = vip_dataframe.sort_values("compound score", ascending=False)
vip_dataframe_unique = vip_dataframe_unique.drop_duplicates(subset=["smiles"], keep="first")


In [35]:
vip_dataframe.to_csv("../data-out-clean/07_31_25_export/MSV000096359_features_vips_classes.csv")